In [7]:
from sklearn.svm import LinearSVR
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

#mean average percent error
def mape(actual, pred): 
    actual_array, pred_array = np.array(actual), np.array(pred)
    return (np.mean(np.abs((actual_array - pred_array) / actual)) * 100)


#Getting the stock data (install yfinance "pip install yfinance")
import yfinance as yf
ticker = yf.download('AAPL', period="5y")
ticker.describe()

#adjusting data to what we need
tickerData = ticker
adjclose = tickerData['Adj Close']
tickerData['Today Adj Close'] = adjclose

tickerData['Adj Close'] = tickerData['Adj Close'].shift(-1)
tickerData = tickerData.dropna()
tickerData = tickerData.drop(['Close'], axis=1)
tickerData = tickerData.rename(columns={"Adj Close" : "Tomorrow Adj Close"})
tickerData = tickerData.drop(['Volume'], axis=1)

attributes = ['Open', 'High', 'Low', 'Tomorrow Adj Close', 'Today Adj Close']
#literally don't use this at all
scale = ['Open', 'High', 'Low', 'Today Adj Close']
scaler = MinMaxScaler()
#feature_transform = scaler.fit_transform(tickerData[scale])
#feature_transform= pd.DataFrame(columns=attributes, data=feature_transform, index=tickerData.index)



#Getting the data for training

#get the labels seperated from the features
train_set, test_set = train_test_split(tickerData, test_size=0.2, random_state=42)
train_set_features = train_set.drop(['Tomorrow Adj Close'], axis=1)
train_set_labels= train_set["Tomorrow Adj Close"].copy()
feature_transform = scaler.fit_transform(train_set_features[scale])
train_prepared = pd.DataFrame(columns=scale, data=feature_transform, index=train_set_features.index)

test_set_features = train_set.drop(['Tomorrow Adj Close'], axis=1)

test_set_labels= train_set["Tomorrow Adj Close"].copy()
   
#linear Regression Model to see how it does
svm_reg = LinearSVR(epsilon=.5)
svm_reg.fit(train_prepared, train_set_labels)

pred = svm_reg.predict(test_set_features)
                                  
#("Predictions:" , lin_reg.predict(test_set_features))
#("Labels:", list(test_set_labels))

#rmse with lin_reg
scores = cross_val_score(svm_reg, train_set_features, train_set_labels, 
                         scoring="neg_mean_squared_error", cv =10) 
rmse = np.sqrt(-scores)
print("Scores:", rmse)
print("Mean:", rmse.mean())
print ("Standared deviation:", rmse.std())
print("MAPE: ", mape(test_set_labels, pred))


print(train_set_labels.describe())                                   


[*********************100%***********************]  1 of 1 completed


/home/klitzlo/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/klitzlo/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/klitzlo/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/klitzlo/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/klitzlo/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, inc

Scores: [1.6119053  1.51707781 1.5193961  1.2539569  1.42279282 1.33683817
 2.02398367 1.74052733 1.88336428 1.91389111]
Mean: 1.6223733477772282
Standared deviation: 0.24642332374180206
MAPE:  13131.164767020244
count    1005.000000
mean       70.479774
std        38.768559
min        25.675816
25%        41.090141
50%        51.864838
75%       110.933540
max       157.869995
Name: Tomorrow Adj Close, dtype: float64


/home/klitzlo/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/klitzlo/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
